# Análise Preditiva de Suicídios

## Descrição do dataset
Os dados foram coletados em forma de questionário através de redes sociais e emails. Alguns direcionados a grupos específicos de apoio (contra depressão). As questões abordadas foram baseadas nesta [pesquisa](https://www.kaggle.com/kingburrito666/the-demographic-rforeveralone-dataset). <br>

<b>Descrição das variáveis:</b><br>
* Q1: "Qual a sua idade? <br>
* Q2: "Qual gênero você mais se identifica?" <br>
* Q3: "Qual sua orientação  sexual?" <br>
* Q4: "Qual o seu nível de renda anual?" <br>
* Q5: "Qual sua etnicidade?" <br>
* Q6: "Como você descreve sua massa corporal?" <br>
* Q7: "Você é virgem?" <br>
* Q8: "Você pagaria para ter uma relação sexual?" <br>
* Q9: "Quantos amigos você tem na vida real?" <br>
* Q10: "Você possui ansiedade/fobia social?" <br>
* Q11: "Você possui algum tipo de diagnóstico de transtorno mental?" <br>
* Q12: "Que tipo de ajuda você quer? <br>
* Q13: "Você já tentou se suicidar?" <br>
* Q14: "Qual seu status empregatício? Atualmente você é..." <br>
* Q15: "Qual seu cargo atual?" <br>
* Q16: "Qual seu nível de educação?" <br>
* Q17: "O que você fez para se sentir melhor consigo mesmo?" <br>
* Q18: "Com quem você mora?" <br>
* Q19: "Seus pais são divorciados?" <br>


In [103]:
import pandas as pd

In [104]:
survey_data = pd.read_csv('survey_data.csv')

In [70]:
df = survey_data.iloc[:, 1:20]
df.columns = [['Q{}'.format(x+1) for x in range(0, 19)]]
df.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19
0,22,Feminino,Bissexual,"R$30,000 - R$39,999",Branco,Peso normal,Não,Não,0,Não,Transtorno Bipolar e Transtornos Relacionados ...,Terapia,Sim,Empregado,Gerente comercial,Superior (graduação) incompleto,Terapia,Com amigo(s),Não
1,28,Masculino,Heterossexual,"R$75,000 - R$99,999",Branco,Peso normal,Não,Não,10,Não,NaN,Não quero ajuda,Não,Empregado,Desenvolvedor de Software,Superior (graduação) incompleto,Indo à academia;Nada,Com amigo(s),Não
2,31,Masculino,Heterossexual,"R$20,000 - R$29,999",Negro,Peso normal,Sim,Sim,9,Não,Transtornos Depressivos (Transtorno Disruptivo...,Não quero ajuda,Não,Empregado,Marketing,Superior (graduação) completo,"Fumo maconha, toco, fumo maconha, transo",Com namorado(a),Não
3,30,Masculino,Heterossexual,"R$1,00 - R$9,999",Branco,Abaixo do peso,Não,Sim,5,Não,"Transtornos de Ansiedade (Fobia Específica, Tr...",Amigos;Terapia,Não,Empregado,Professor,Pós-graduação completo,Dieta;Indo à academia;Procedimento cosmético,"Com meus pais (somente pai, somente mãe ou ambos)",Não
4,25,Feminino,Heterossexual,"R$1,00 - R$9,999",Branco,Peso normal,Não,Não,5,Não,Transtorno Bipolar e Transtornos Relacionados ...,Terapia,Sim,Autônomo,NaN,Superior (graduação) completo,Indo à academia;Terapia,"Com meus pais (somente pai, somente mãe ou ambos)",Não


## 1. Pré-processamento